In [ ]:
# prompt: import drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews_supplements.csv')
df

,rating,title,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,date,time
0,4,B Complex in gel cap form,I bought this along with Vit C in gel cap form...,B00012ND5G,B00012ND5G,AGDVFFLJWAQ3ULNNKF4LXID2RVSQ,2009-12-11 00:37:33,1,True,2009-12-11,00:37
1,5,Five Stars,great product,B00013Z0ZQ,B00013Z0ZQ,AG3BSKXHDGP6E3EGQD2SXCK6KFQQ,2015-01-04 03:11:26,0,True,2015-01-04,03:11
2,5,Five Stars,Came as expectedly,B00013Z0ZQ,B00013Z0ZQ,AHG2WKFD4LXPC46WWC6JMQGX52JA,2015-09-27 19:15:33,0,True,2015-09-27,19:15
3,5,Vitamin Shoppe Dry Vitamin A,Excellent Product ..... Fast Delivery ....... ...,B00013Z1KA,B00013Z1KA,AEOF7RT3AC4ACRX5HGIP2V3BNIHA,2019-02-09 19:33:16.911,0,True,2019-02-09,19:33
4,5,Un producto que compro regularmente,Es muy buena vitamina,B00013Z1KA,B00013Z1KA,AGW2WETWQRL2PKUGTL2LU7IJ2BPQ,2022-07-25 14:11:10.936,0,True,2022-07-25,14:11
...,...,...,...,...,...,...,...,...,...,...,...
16666,5,Love!,"I cannot say enough about this gel! Hands, arm...",B0009VO8EO,B0C7RQ35K7,AFUCOHHPFBX6RHHUW7OGFHSBQ5KQ,2022-10-26 12:23:51.529,0,True,2022-10-26,12:23
16667,5,Used to use this!,Love this gel for skin. Used to use Banana Bo...,B0009VO8EO,B0C7RQ35K7,AEKMUWW2ZDLOOKNL6QFX7DRKCNEQ,2022-12-01 16:21:03.209,0,True,2022-12-01,16:21
16668,5,works great,Easy to use and it really works.,B0009VO8EO,B0C7RQ35K7,AF3GLUKTNSQN4YYV6II2JU3LHS2Q,2023-01-11 19:46:27.719,0,True,2023-01-11,19:46
16669,4,That this product is unbelievably perfect,I had surgery and both of my doctors recommend...,B0009VO8EO,B0C7RQ35K7,AECDMI6JKOAPXNCYCWSKQQPEO6YQ,2023-03-17 11:19:10.310,0,True,2023-03-17,11:19


In [ ]:
df = df[['rating', 'title',	'text',	'asin',	'parent_asin'	,'user_id',	'helpful_vote',	'verified_purchase']]

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)

from typing import Type, List, Optional
import csv

from pydantic import BaseModel
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator
from langchain_core.output_parsers import PydanticOutputParser

In [ ]:
class Review_record(BaseModel):
    rating: int
    title : str
    text: str
    asin: str
    parent_asin: str
    user_id: str
    helpful_vote: int
    verified_purchase: bool

hr_examples =[
    {
        "example": """rating: 4, title: B Complex in gel cap form, text: 'I bought this along with Vit C in gel cap form because a doctor explained on his website that he tests the blood of his patients regularly for the levels of various supplements he recommends and he claimed that pills seemed to result in extremely poor and uneven levels in the body. He also claimed gel caps were much better compared to pills in terms of a more even and reliable absorption into the blood stream.', asin: 'B00012ND5G', parent_asin: 'B00012ND5G', user_id: 'AGDVFFLJWAQ3ULNNKF4LXID2RVSQ', helpful_vote: 1, verified_purchase: True"""
    },
    {
        "example": """rating: 5, title: Five Stars, text: great product, asin: B00013Z0ZQ, parent_asin: B00013Z0ZQ, user_id: AG3BSKXHDGP6E3EGQD2SXCK6KFQQ, helpful_vote: 0, verified_purchase: True"""
    },
    {
        "example": """rating: 5, title: Five Stars, text: Came as expectedly, asin: B00013Z0ZQ, parent_asin: 'B00013Z0ZQ', user_id: AHG2WKFD4LXPC46WWC6JMQGX52JA, helpful_vote: 0, verified_purchase: True"""
    },
    {
        "example": """rating: 5, title: Vitamin Shoppe Dry Vitamin A, text: Excellent Product ..... Fast Delivery ....... Will Buy From Again ........ A+A+A+A+A+, asin: B00013Z1KA, parent_asin: B00013Z1KA, user_id: AEOF7RT3AC4ACRX5HGIP2V3BNIHA, helpful_vote: 0, verified_purchase: True"""
    },
    {
        "example": """rating: 1, title: One Star, text: Not effective, asin: B00013Z0ZQ, parent_asin: B00013Z0ZQ, user_id: AHG2WKFD4LXPC46WWC6JMQGX52JA, helpful_vote: 0, verified_purchase: True"""
    }
]

prompt_template = FewShotPromptTemplate(
    prefix = "This is a test about generating synthetic data about {subject}. Examples below:",
    examples = hr_examples,
    suffix = "Now you generate synthetic data about {subject}. Make sure to {extra}:",
    input_variables = ["subject", "extra"],
    example_prompt = PromptTemplate(input_variables=["example"], template="{example}"),
)

synthetic_data_generator = create_openai_data_generator(
    output_schema=Review_record,
    llm=ChatOpenAI(model = "gpt-4o", temperature=1, api_key="enter_api_key"),
    prompt=prompt_template,
)

synthetic_results = None

synthetic_results = synthetic_data_generator.generate(
    subject="review_data",
    extra="Make the name something you would usually not pick",
    runs=2,
)

In [ ]:
synthetic_results

[Review_record(rating=3, title='Unexpected Results, but Satisfied', text="I started using this product a month ago hoping for quick results. Although the effects were more subtle than anticipated, I've noticed a gradual improvement in energy throughout my day. Worth trying for anyone curious.", asin='B0001EL777', parent_asin='B0001EL777', user_id='ZZG128UOFLR3NIOWU9GEQ8TYZ5XQ', helpful_vote=2, verified_purchase=True),
 Review_record(rating=2, title='Underwhelming Experience', text="I had high hopes for this product based on the reviews, but unfortunately, it didn't live up to my expectations. It barely made a difference.", asin='B00013Z0ZQ', parent_asin='B00013Z0ZQ', user_id='AFX7PLOYWAEICJPLR8ZXU3R5ODJD', helpful_vote=1, verified_purchase=False)]

In [ ]:
df = pd.DataFrame(synthetic_results)

In [ ]:
df.to_csv('synthetic_data.csv')